# Pipeline de inferencia

## Entorno y rutas
Este bloque configura el entorno de Python y ajusta las rutas para que los imports de módulos internos funcionen correctamente en cualquier entorno (local o CI/CD).

In [5]:
# === Configuración de rutas para importación de módulos internos ===
# Añadimos el directorio raíz del proyecto al sys.path para poder importar módulos locales (por ejemplo, src, config, etc.)
import sys
from pathlib import Path

# Si necesitas importar módulos de src, descomenta la siguiente línea:
sys.path.append(str(Path().resolve().parent / 'src'))

In [6]:
# === Recarga automática de módulos (útil en desarrollo interactivo) ===
%reload_ext autoreload
%autoreload 2

## Ajuste de sys.path para imports
Este bloque garantiza que el directorio raíz del proyecto esté en sys.path, permitiendo importar módulos personalizados sin errores.

In [7]:
# === Asegura que el directorio raíz está en sys.path ===
# Esto es importante para que los imports funcionen correctamente en cualquier entorno (local, CI/CD, etc.)
import sys
from pathlib import Path

project_root = Path().resolve().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

## Conexión, exploración y acceso a Hopsworks
Este bloque es opcional y permite inspeccionar los datos disponibles en el feature store de Hopsworks. Útil para depuración y validación.

In [ ]:
# === Conexión y exploración de feature groups en Hopsworks ===
# Este bloque conecta con Hopsworks y lista los feature groups disponibles para el nombre dado.
import hopsworks
import pandas as pd
import config

FG_NAME = "times_series_bolleria_feature_group"
FV_NAME = "times_series_bolleria_feature_view"
FV_VERSION = 1

# Conectar con el proyecto de Hopsworks y al feature store
project = hopsworks.login(project=config.HOPSWORKS_PROJECT_NAME, api_key_value=config.HOPSWORKS_API_KEY)

# Obtener el feature store
feature_store = project.get_feature_store()

# Listar los feature groups existentes para FG_NAME
feature_groups = feature_store.get_feature_groups(name=FG_NAME)
print(f"Feature groups encontrados: {len(feature_groups)}")
for fg in feature_groups:
    print(f" - {fg.name}")

DEBUG ENV HOPSWORKS_PROJECT_NAME: fleca_mlops
DEBUG ENV PATH: C:\Workspace\mlops_fleca_project\.env
2025-09-07 12:57:58,843 INFO: Initializing external client
2025-09-07 12:57:58,844 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-09-07 12:57:58,844 INFO: Base URL: https://c.app.hopsworks.ai:443




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (4.2) by running 'pip install hopsworks==4.2.*'


2025-09-07 12:58:00,101 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1242272

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1242272
Feature groups encontrados: 1
 - times_series_bolleria_feature_group
Feature groups encontrados: 1
 - times_series_bolleria_feature_group


### Exploración de datos en Hopsworks
Este bloque conecta con Hopsworks y permite explorar los feature groups y feature views disponibles para validar la estructura de los datos.

In [9]:
# === FEATURE GROUPS ===

#  Para cada Feature Group, saca numero de filas y rango de timestamp
for fg in feature_groups:
    print(f"Feature group: {fg.name}")
    df = fg.read()
    print(f" - Número de filas: {len(df)}")
    if not df.empty:
        print(f" - Rango de week_start: {df['week_start'].min()} a {df['week_start'].max()}")
    else:
        print(" - No hay datos en este feature group.")

Feature group: times_series_bolleria_feature_group
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.55s) 
 - Número de filas: 135
 - Rango de week_start: 2023-01-02 00:00:00+00:00 a 2025-08-25 00:00:00+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.55s) 
 - Número de filas: 135
 - Rango de week_start: 2023-01-02 00:00:00+00:00 a 2025-08-25 00:00:00+00:00


In [10]:
# === FEATURE VIEWS ===

# Listas los features views
feature_views = feature_store.get_feature_views(name=FV_NAME)
# Filtrar por versión
feature_views = [fv for fv in feature_views if fv.version == FV_VERSION]
print(f"Feature views encontrados: {len(feature_views)}")
for fv in feature_views:
    print(f" - {fv.name}")
    print(f" - Version: {fv.version}")

# Para cada feature view saca el número de filas y rango de timestamp
batch_data = []
for fv in feature_views:
    df = fv.get_batch_data()
    batch_data.append(df)

# Imprimir información sobre los datos de cada feature view y las columnas
for i, df in enumerate(batch_data):
    print(f"Feature view {feature_views[i].name}:")
    print(f" - Número de filas: {len(df)}")
    if not df.empty:
        print(f" - Rango de week_start: {df['week_start'].min()} a {df['week_start'].max()}")
        print(f" - Columnas: {list(df.columns)}")
    else:
        print(" - No hay datos en este feature view.")

Feature views encontrados: 1
 - times_series_bolleria_feature_view
 - Version: 1
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.51s) 
Feature view times_series_bolleria_feature_view:
 - Número de filas: 135
 - Rango de week_start: 2023-01-02 00:00:00+00:00 a 2025-08-25 00:00:00+00:00
 - Columnas: ['familia', 'base_imponible', 'is_summer_peak', 'is_easter', 'week_start']
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.51s) 
Feature view times_series_bolleria_feature_view:
 - Número de filas: 135
 - Rango de week_start: 2023-01-02 00:00:00+00:00 a 2025-08-25 00:00:00+00:00
 - Columnas: ['familia', 'base_imponible', 'is_summer_peak', 'is_easter', 'week_start']


## Batch Score del Feature View

In [12]:
from datetime import timedelta
import pandas as pd

# Lectura batch cruda de feature views
feature_views= feature_store.get_feature_view(name=FV_NAME, version=FV_VERSION)
ts_df = feature_views.get_batch_data(
    start_date=None,
    end_date=None,
    features=None,
    )

# Ordenar por la columna de fecha antes de visualizar
ts_df = ts_df.sort_values('week_start').reset_index(drop=True)
ts_df.tail(5)

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.55s) 
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.55s) 


,familia,base_imponible,is_summer_peak,is_easter,week_start
130,BOLLERIA,1408.04,1,0,2025-07-28 00:00:00+00:00
131,BOLLERIA,1609.03,1,0,2025-08-04 00:00:00+00:00
132,BOLLERIA,1782.56,1,0,2025-08-11 00:00:00+00:00
133,BOLLERIA,1741.72,1,0,2025-08-18 00:00:00+00:00
134,BOLLERIA,1402.02,1,0,2025-08-25 00:00:00+00:00


## Features (lags, variables exógenas) y target
En este bloque se generan los lags y variables exógenas necesarias para la predicción, asegurando que los datos estén en el formato correcto para el modelo.

## Modelo
Este bloque recupera el modelo entrenado desde el model registry de Hopsworks para realizar la predicción.

In [ ]:
from src.model import transformar_features_target

# Procesar datos usando la función mejorada que acepta tuplas directamente
ts_df = transformar_features_target(
    ts_df,
    lags_list=[1, 52], 
    columna_target='base_imponible',
    cols_exogenas=['is_easter', 'is_summer_peak'],
    periodos_adelante=1,
    eliminar_nulos=True,
    return_format='dataframe'  # Obtenemos un único DataFrame con features y target
)

# Resetear el índice
ts_df = ts_df.reset_index(drop=True)

# Mostrar las primeras filas del DataFrame transformado
ts_df.head(5)

In [14]:
import joblib
# Cargar el modelo del model registry
## se obtiene del config

MODEL_NAME = "fleca_bolleria_predictor_next_week"
MODEL_VERSION = 4
MODEL_FILE ='xgboost_hopsworks.pkl'


model_registry = project.get_model_registry()
model = model_registry.get_model(name=MODEL_NAME, version=MODEL_VERSION)    
from pathlib import Path

model_dir = model.download()
model = joblib.load(Path(model_dir) / MODEL_FILE)

print(f'Modelo {MODEL_NAME} V{MODEL_VERSION} cargado correctamente')

Downloading: 0.000%|          | 0/224774 elapsed<00:00 remaining<?

Modelo fleca_bolleria_predictor_next_week V4 cargado correctamente


## Verificación features

Antes de realizar la predicción, es fundamental comprobar que las columnas generadas tras la ingeniería de features (`transformar_features_target`) coinciden exactamente con las que espera el modelo entrenado.  
Este bloque imprime las columnas disponibles en el DataFrame, los nombres de los features que espera el modelo y realiza una comparación uno a uno, indicando si falta alguno o si todos están presentes.  
Además, se verifica la existencia de columnas relacionadas con los lags del target, lo que ayuda a depurar posibles errores de desalineación entre entrenamiento e

In [15]:
# Verificar qué columnas tenemos después de transformar_features_target
print("Columnas después de transformar_features_target:")
print(ts_df.columns.tolist())

print("\nFeatures que espera el modelo:")
if 'model' in globals():
    print(model.feature_names_in_)
else:
    print("❌ El modelo no está definido. Por favor, ejecuta la celda que carga el modelo antes de continuar.")

print("\nComparación:")
for feature in model.feature_names_in_:
    if feature in ts_df.columns:
        print(f"✅ {feature}: EXISTE")
    else:
        print(f"❌ {feature}: NO EXISTE")
        
# Verificar si existen las versiones con 'target_lag'
target_lags = [col for col in ts_df.columns if 'target_lag' in col]
print(f"\nColumnas target_lag encontradas: {target_lags}")

Columnas después de transformar_features_target:
['base_imponible_lag1', 'base_imponible_lag52', 'is_easter', 'is_summer_peak', 'week_start', 'target']

Features que espera el modelo:
['base_imponible_lag1' 'base_imponible_lag52' 'is_easter' 'is_summer_peak']

Comparación:
✅ base_imponible_lag1: EXISTE
✅ base_imponible_lag52: EXISTE
✅ is_easter: EXISTE
✅ is_summer_peak: EXISTE

Columnas target_lag encontradas: []


In [16]:
# Imprimir los nombres de los features que el modelo espera 
model.feature_names_in_


array(['base_imponible_lag1', 'base_imponible_lag52', 'is_easter',
       'is_summer_peak'], dtype='<U20')

In [17]:
# Limpiar columnas innecesarias
features = ts_df.drop(columns=['week_start'])

## Inferencia

###  Predicción de la próxima semana
En este bloque se realiza la predicción de ventas para la próxima semana utilizando el modelo cargado y los features generados.

In [18]:
# Predicción directa para la próxima semana (sin autoregresión)
from datetime import timedelta

print("=== PREDICCIÓN DIRECTA DE LA PRÓXIMA SEMANA ===")

# Tomar la última fila de datos históricos
ultimo_lunes = ts_df['week_start'].max()
last_row = ts_df.iloc[-1].copy()
feature_names = model.feature_names_in_

# Calcular la fecha de la próxima semana
fecha_siguiente = ultimo_lunes + timedelta(days=7)
last_row['week_start'] = fecha_siguiente

# Preparar los features para el modelo
X = last_row[feature_names].values.reshape(1, -1)
prediccion = model.predict(X)[0]

print(f"Fecha predicha: {fecha_siguiente.date()}")
print(f"Predicción base_imponible: {prediccion:.2f}")

# Mostrar resultado en DataFrame
resultado = pd.DataFrame({
    'week_start': [fecha_siguiente],
    'predicted_base_imponible': [prediccion]
})
print(resultado)


=== PREDICCIÓN DIRECTA DE LA PRÓXIMA SEMANA ===
Fecha predicha: 2025-08-25
Predicción base_imponible: 1249.53
                 week_start  predicted_base_imponible
0 2025-08-25 00:00:00+00:00               1249.527588
Fecha predicha: 2025-08-25
Predicción base_imponible: 1249.53
                 week_start  predicted_base_imponible
0 2025-08-25 00:00:00+00:00               1249.527588


### Subida de la predicción a Hopsworks
Este bloque inserta la predicción semanal en el feature group y crea (o actualiza) el feature view correspondiente en Hopsworks.

In [19]:
# Subir predicción semanal (variable 'resultado') al feature group y feature view de Hopsworks
from hsfs.feature_group import FeatureGroup
from hsfs.feature_view import FeatureView
import logging

# Configuración básica de logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger('feature_group_creation')

PRED_FG_NAME = "bolleria_predicciones_feature_group"
PRED_FG_VERSION = 1

# Usar la variable 'resultado' como DataFrame de predicción
df_pred = resultado.copy()  # Debe tener columnas 'week_start' y 'predicted_base_imponible'

# Mostrar los datos que se van a insertar
print("DataFrame que se va a insertar:")
print(df_pred)

# 1. Primero intentamos crear el feature group explícitamente
try:
    # Si ya existe, esto lanzará una excepción que capturaremos después
    fg_pred = feature_store.create_feature_group(
        name=PRED_FG_NAME,
        version=PRED_FG_VERSION,
        description="Predicciones semanales de base_imponible para bolleria",
        primary_key=["week_start"],
        online_enabled=False,
        statistics_config={"enabled": True},
        event_time='week_start'
    )
    logger.info(f"Feature group '{PRED_FG_NAME}' creado correctamente.")
    
    # Al crear un nuevo feature group, necesitamos hacer insert con el DataFrame
    insert_job = fg_pred.insert(df_pred, write_options={'wait_for_job': True})
    logger.info(f"Datos insertados en el feature group recién creado. Job: {insert_job}")
    
except Exception as create_error:
    logger.warning(f"No se pudo crear el feature group: {create_error}")
    logger.info("Intentando obtener el feature group existente...")
    
    # 2. Si el feature group ya existe, lo obtenemos
    try:
        fg_pred = feature_store.get_feature_group(
            name=PRED_FG_NAME,
            version=PRED_FG_VERSION
        )
        logger.info(f"Feature group existente recuperado: {fg_pred.name} v{fg_pred.version}")
        
        # Insertamos los nuevos datos en el feature group existente
        insert_job = fg_pred.insert(df_pred, write_options={'wait_for_job': True})
        logger.info(f"Datos insertados en el feature group existente. Job: {insert_job}")
        
    except Exception as get_error:
        logger.error(f"Error al recuperar el feature group: {get_error}")
        logger.error("No se pudo ni crear ni recuperar el feature group.")
        import traceback
        traceback.print_exc()
        fg_pred = None

# 3. Verificación final
if fg_pred is not None:
    print(f"✅ Feature group operativo: {fg_pred.name} (versión {fg_pred.version})")
    
    # 4. (Opcional) Crear el feature view si no existe
    try:
        # Primero intentar recuperar
        try:
            fv_pred = feature_store.get_feature_view(name=PRED_FG_NAME, version=PRED_FG_VERSION)
            print(f"✅ Feature view '{PRED_FG_NAME}' ya existe.")
        except:
            # Si no existe, crear uno nuevo
            query = fg_pred.select_all()
            fv_pred = feature_store.create_feature_view(
                name=PRED_FG_NAME,
                version=PRED_FG_VERSION,
                query=query,
                description="Vista de predicciones semanales de bolleria"
            )
            print("✅ Feature view creado correctamente.")
    except Exception as fv_error:
        print(f"⚠️ Error con el feature view: {fv_error}")
else:
    print("❌ Error: No se pudo operar con el feature group de predicciones.")

DataFrame que se va a insertar:
                 week_start  predicted_base_imponible
0 2025-08-25 00:00:00+00:00               1249.527588
2025-09-07 12:58:15,844 INFO: Feature group 'bolleria_predicciones_feature_group' creado correctamente.
2025-09-07 12:58:16,009 WARNING: No se pudo crear el feature group: Metadata operation error: (url: https://c.app.hopsworks.ai/hopsworks-api/api/project/1242272/featurestores/1224799/featuregroups). Server response: 
HTTP code: 400, HTTP reason: Bad Request, body: b'{"errorCode":270089,"usrMsg":"project: fleca_mlops, featurestoreId: 1224799","errorMsg":"The feature group you are trying to create does already exist."}', error code: 270089, error msg: The feature group you are trying to create does already exist., user msg: project: fleca_mlops, featurestoreId: 1224799
2025-09-07 12:58:16,010 INFO: Intentando obtener el feature group existente...
2025-09-07 12:58:16,009 WARNING: No se pudo crear el feature group: Metadata operation error: (url: htt

Uploading Dataframe: 100.00% |██████████| Rows 1/1 | Elapsed Time: 00:00 | Remaining Time: 00:00



Launching job: bolleria_predicciones_feature_group_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1242272/jobs/named/bolleria_predicciones_feature_group_1_offline_fg_materialization/executions
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1242272/jobs/named/bolleria_predicciones_feature_group_1_offline_fg_materialization/executions
2025-09-07 12:58:30,223 INFO: Waiting for execution to finish. Current state: INITIALIZING. Final status: UNDEFINED
2025-09-07 12:58:30,223 INFO: Waiting for execution to finish. Current state: INITIALIZING. Final status: UNDEFINED
2025-09-07 12:58:33,406 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-09-07 12:58:33,406 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-09-07 12:58:36,591 INFO: Waiting for execution to finish. Current state: RUNNING. Fin

## Verificación del feature group y creación del feature view

Después de crear el feature group, verificamos su existencia y creamos un feature view asociado.

In [20]:
# Verificar que el feature group existe y crear feature view
import logging
from hsfs.feature_group import FeatureGroup
from hsfs.feature_view import FeatureView

# Configuración de logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger('feature_view_creation')

# Definir constantes
PRED_FG_NAME = "bolleria_predicciones_feature_group"
PRED_FG_VERSION = 1
PRED_FV_NAME = "bolleria_predicciones_feature_view"
PRED_FV_VERSION = 1

# 1. Verificar que el feature group existe
try:
    fg_pred = feature_store.get_feature_group(name=PRED_FG_NAME, version=PRED_FG_VERSION)
    print(f"✅ Feature group encontrado: {fg_pred.name} (versión {fg_pred.version})")
    
    # Mostrar los datos actuales en el feature group
    print("\nDatos actuales en el feature group:")
    df_fg = fg_pred.read()
    print(df_fg)
    
    # 2. Crear feature view
    print("\nCreando feature view...")
    
    try:
        # Primero intentamos recuperar el feature view si existe
        try:
            fv_pred = feature_store.get_feature_view(name=PRED_FV_NAME, version=PRED_FV_VERSION)
            print(f"✅ Feature view existente: {fv_pred.name} (versión {fv_pred.version})")
        except:
            # Si no existe, crear uno nuevo
            query = fg_pred.select_all()  # Crear la query seleccionando todas las columnas
            
            print("Creando nuevo feature view...")
            fv_pred = feature_store.create_feature_view(
                name=PRED_FV_NAME,
                version=PRED_FV_VERSION,
                query=query,
                description="Vista de predicciones semanales de bolleria"
            )
            print(f"✅ Nuevo feature view creado: {fv_pred.name} (versión {fv_pred.version})")
        
        # Verificar datos en el feature view
        print("\nVerificando datos en el feature view:")
        df_fv = fv_pred.get_batch_data()
        print(df_fv)
        
    except Exception as fv_error:
        print(f"❌ Error al crear o recuperar el feature view: {fv_error}")
        logger.error(f"Error detallado: {fv_error}")
        import traceback
        traceback.print_exc()

except Exception as e:
    print(f"❌ Error al recuperar el feature group: {e}")
    logger.error(f"Error detallado: {e}")
    import traceback
    traceback.print_exc()

✅ Feature group encontrado: bolleria_predicciones_feature_group (versión 1)

Datos actuales en el feature group:
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.07s) 
                 week_start  predicted_base_imponible
0 2025-08-11 00:00:00+00:00               1310.636108
1 2025-08-18 00:00:00+00:00               1817.525269
2 2025-08-25 00:00:00+00:00               1249.527588

Creando feature view...
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.07s) 
                 week_start  predicted_base_imponible
0 2025-08-11 00:00:00+00:00               1310.636108
1 2025-08-18 00:00:00+00:00               1817.525269
2 2025-08-25 00:00:00+00:00               1249.527588

Creando feature view...
✅ Feature view existente: bolleria_predicciones_feature_view (versión 1)

Verificando datos en el feature view:
✅ Feature view existente: bolleria_predicciones_feature_view (versión 1)

Verificando datos en el feature view:
Finished:

# Visualización de las predicciones

Una vez tenemos las predicciones almacenadas en el feature store, podemos visualizarlas para comprender mejor las tendencias.

In [21]:
# Visualización de datos históricos y predicciones
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime, timedelta

# 1. Obtener datos históricos del feature view original
try:
    # Obtenemos los datos históricos
    fv_historico = feature_store.get_feature_view(name=FV_NAME, version=FV_VERSION)
    df_historico = fv_historico.get_batch_data()
    df_historico = df_historico.sort_values('week_start')
    
    # 2. Obtenemos las predicciones del feature view de predicciones
    fv_pred = feature_store.get_feature_view(name=PRED_FV_NAME, version=PRED_FV_VERSION)
    df_pred = fv_pred.get_batch_data()
    df_pred = df_pred.sort_values('week_start')
    
    # 3. Preparar datos para visualización
    # Convertir a formato datetime si es necesario
    if not pd.api.types.is_datetime64_any_dtype(df_historico['week_start']):
        df_historico['week_start'] = pd.to_datetime(df_historico['week_start'])
    
    if not pd.api.types.is_datetime64_any_dtype(df_pred['week_start']):
        df_pred['week_start'] = pd.to_datetime(df_pred['week_start'])
    
    # 4. Visualización con Plotly
    fig = go.Figure()
    
    # Añadir datos históricos
    fig.add_trace(go.Scatter(
        x=df_historico['week_start'],
        y=df_historico['base_imponible'],
        mode='lines+markers',
        name='Histórico',
        line=dict(color='blue', width=2)
    ))
    
    # Añadir predicciones
    fig.add_trace(go.Scatter(
        x=df_pred['week_start'],
        y=df_pred['predicted_base_imponible'],
        mode='lines+markers',
        name='Predicciones',
        line=dict(color='red', width=2, dash='dash')
    ))
    
    # Configurar diseño
    fig.update_layout(
        title='Ventas Históricas y Predicciones para Bollería',
        xaxis_title='Fecha',
        yaxis_title='Base Imponible (€)',
        legend_title='Tipo de Dato',
        height=600,
        template='plotly_white'
    )
    
    # Mostrar gráfico
    fig.show()
    
except Exception as e:
    print(f"❌ Error al visualizar los datos: {e}")
    import traceback
    traceback.print_exc()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.62s) 
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.62s) 
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.54s) 
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.54s) 


# FIN